"1.Standard GD and Momentum"中罗列了重要的数学记号,并讲了标准梯度下降法(GD, Gradient Descent)和Momentum机制.今天我们换个角度来解释Momentum,然后介绍RMSProp(Root Mean Square Propagation)和AdaM(Adaptive Moment Algo).




# 换角度看Momentum  

第一篇中讲到带有Momentum的GD的迭代方法为  
$\Delta x_t =  - \eta* g_t + \rho * \Delta x_{t-1} $

这个式子可以帮助我们更好的理解“momentum”.  
不过它也可以改写为  
$$tmp_{t+1} = g_t + \rho * tmp_t  \\ ( \text{记住}tmp_t= - \frac{1}{\eta} * \Delta x_{t-1} ) \\ \Delta x_t =  - \eta * tmp_{t+1} $$
进一步改写为
$$v_{t+1} =  \beta_1 * v_t +  (1 - \beta_1) * g_t  $$  
$$ \Delta x_t = - \eta_2 * v_{t+1} $$  

两者完全等价,只需要 

$$ \beta_1 = \frac{\rho}{1 + \rho},  \eta_2 = \eta *(1 + \rho)  $$  

我们扩展下
$ v_{t+1}   $

$ = \beta_1 * v_t + (1 - \beta_1) * g_t   $

$= (\beta_1)^2 * v_{t-1} + \beta_1 * (1-\beta_1) * g_{t-1} +  (1 - \beta_1) * g_t  $

$=  (\beta_1)^3 * v_{t-2} +  (\beta_1)^2 * (1-\beta_1) * g_{t-2} + \beta_1 * (1-\beta_1) * g_{t-1} +  (1 - \beta_1) * g_t  $ 

$= (\beta_1)^{t+1} * v_0 +  (1 - \beta_1 ) * \sum_{\tau=0}^t (\beta_1)^{\tau}  * g_{t-\tau} $

因为 $ (1 - \beta_1 ) * \sum_{\tau=0}^t (\beta_1)^{\tau} = 1 - (\beta_1)^{t+1} $. 
设 $v_0 = 0 $
我们就会发现Momentum每次迭代的方向是当次迭代为止所有梯度$\{g_{\tau} \}_{\tau=0}^t \cup \{0\}$. 的指数加权平均!做过时序模型的一定不陌生.而均值在统计学中叫做一阶距.

# RMSProp (Root Mean Square Propagation)

RMSProp来源于[Hinton Coursera神经网络课程的课后作业](https://www.coursera.org/learn/neural-networks/home/welcome)和Momentum一样,RMSProp也可以从多个角度理解.


首先给出更新公式
$$w_{t+1} = \beta_2 * w_t + (1 - \beta_2) * (g_t)^2 $$
$$\Delta x_t = -\eta * \frac{1}{\sqrt{w_{t+1}}+ \epsilon} * g_t $$
其中
- $\sqrt{w_{t+1}}  $是 $w_{t+1} $向量各个维度(element-wise)的平方根,比如 $\sqrt{(1, 4, 9)} = (1, 2, 3) $ 

- $\epsilon $是很小的常数,用来稳定数值计算(当 $\sqrt{w_{t+1}}$  某一维度很小时)

类似之前对 $v_{t+1} $的展开,我们同样可以得到
$$ w_{t+1} = (\beta_2)^{t+1} * w_0 +  (1 - \beta_2 ) * \sum_{\tau=0}^t (\beta_2)^{\tau}  * (g_{t-\tau})^2 $$
也就是 $w_{t+1}$ 是直到当前迭代为止所有梯度的平方的平均.这也解释了为什么叫做Root Mean Square.

![](RMSProp.jpg)

RMSProp达到的效果是对幅度的自适应调整:
- 之前更新幅度大的维度上更新幅度变小
- 之前更新幅度不大的维度上更新幅度相对大些

那为什么这个效果很有效,在神经网络优化中脱颖而出呢?据说很大一部分原因是  
每次用mini-batch迭代,不同的参数梯度可能相差几个量级,通过梯度的平方的指数加权平均,可以有效得不同参数在多次迭代中的量级差,效果类似于批标准化(BN, Batch Normalization,每一次mini-batch迭代,该批每个参数都先经过仿射变换使得均值为0,方差为1).

此外RMSProp自称在non-stationary的环境下表现很好.因为梯度的平方采用了指数加权平均.而所谓non-stationary环境就是随机变量的统计量随时间改变的性质.

# AdaM(Adaptive Moment estimates)  

AdaM(Adaptive Moment estimates)是一种结合了Momentum和RMSProp的自适应学习率方法.从它的迭代可以看出
![v的更新](v.png)

![w的更新](w.jpg)
$$v^{corrected}_{t+1} = \frac{v_{t+1}}{1-\beta^{t+1}_1} $$

$$w^{corrected}_{t+1} = \frac{w_{t+1}}{1-\beta^{t+1}_2} $$
$$\Delta x_t = - \eta * \frac{v^{corrected}_{t+1}}{\sqrt{w^{corrected}_{t+1}} + \epsilon} $$

其中 $v^{corrected}_{t+1} $为矫正后的梯度的一阶矩(均值), $w^{corrected}_{t+1} $为矫正后的梯度的二阶矩(未中心化的方差),所以叫做Adaptive Moment estimates.  
 
如果说Momentum是用一阶矩来估计梯度方向,RMSProp用二阶矩来估计更新幅度,那AdaM就是结合一阶矩和二阶矩来同时估计梯度方向和更新幅度.

![RMSProp vs AdaM](RMSProp_AdaM.jpg)